# Sensitivity analysis on previous work for 1s, 1.5s and 2s data

training data from 5 people, testing
from person A, with spectrogram ”window”, or duration of 1.5, 1 and
2 seconds respectively and 0.1 second ”shift"

In [1]:
import numpy as np
import os
import sys
from six.moves import cPickle
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

#NEW--------------------
import sys
from sklearn.metrics import classification_report,confusion_matrix
import csv
from keras.models import load_model
#-----------------------

Using TensorFlow backend.


In [0]:
# Needed as originally code was for theano backend but now using tensor flow
from keras import backend as K
K.set_image_dim_ordering('th')

In [3]:
# Check if using CoLab or local pc and set file path accordingly
if (os.getcwd() == '/content'):
    from google.colab import drive
    drive.mount('/content/gdrive')
    FILE_PATH = '/content/gdrive/My Drive/Level-4-Project/data/'
    
else:
    FILE_PATH = os.getcwd()

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
def load_batch(fpath, label_key='labels'):
    f = open(fpath, 'rb')
    if sys.version_info < (3,):
        d = cPickle.load(f)
    else:
        d = cPickle.load(f, encoding="bytes")
        # decode utf8
        for k, v in d.items():
            # added check as otherwise tries to decode a string
            if type(k) is not str:
                del(d[k])
                d[k.decode("utf8")] = v
    f.close()
    data = d["data"]
    labels = d[label_key]

    data = data.reshape(data.shape[0], 1, 75, 75)
    return data, labels

In [0]:
def load_data(data_folder):
    path = FILE_PATH + data_folder

    nb_train_samples = 34720

    X_train = np.zeros((nb_train_samples, 1, 75, 75), dtype="uint8")
    y_train = np.zeros((nb_train_samples,), dtype="uint8")

    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        data, labels = load_batch(fpath)
        X_train[(i-1)*6944:i*6944, :, :, :] = data
        y_train[(i-1)*6944:i*6944] = labels

    fpath = os.path.join(path, 'test_batch')
    X_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    return (X_train, y_train), (X_test, y_test)


In [0]:
def make_model():
    # Updated to use new version of keras
    model = Sequential()

    model.add(Convolution2D(64, (5, 5), padding='same', input_shape=(1, img_rows, img_cols), activation='relu'))
    model.add(Convolution2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Convolution2D(128, (5, 5), padding='same', activation='relu'))
    model.add(Convolution2D(128, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes, activation='softmax'))
    return model

In [0]:
data_folders = ['1s', '1_5s', '2s']

In [0]:
batch_size = 100
nb_classes = 7
nb_epoch = 100
nb_epoch = 2 # originally 100
data_augmentation = False

# input image dimensions
img_rows, img_cols = 75, 75
# the CIFAR10 images are RGB

In [12]:
for data_folder in data_folders:
    # the data, shuffled and split between train and test sets
    (X_train, y_train), (X_test, y_test) = load_data(data_folder)
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    
    model = make_model()
    
    # let's train the model using SGD + momentum (how original).
    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    if not data_augmentation:
        print('Not using data augmentation.')
        hist = model.fit(X_train, Y_train,
                  batch_size=batch_size,
                  epochs=nb_epoch,
                  validation_data=(X_test, Y_test),
                  shuffle=True)


        #NEW--------------------
        y_pred = model.predict_classes(X_test)
        print(y_pred)
        target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
    #     sys.stdout = open('/home/aleksandar/new_sets/1_5s/results/report_1_5s_64_128_.txt', "w")
        print("CLASSIFICATION REPORT:")
        print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
        print ("\n")
        print("CONFUSION MATRIX:")
        print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

    #     with open('/home/aleksandar/new_sets/1_5s/results/graphs_1_5s_64_128.csv', "w") as f:
    #        w = csv.writer(f)
    #        train_loss = hist.history['loss']
    #        val_loss = hist.history['val_loss']
    #        train_acc = hist.history['acc']
    #        val_acc = hist.history['val_acc']
    #        helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
    #        w.writerow("train_loss, train_acc, val_loss, val_acc")
    #        for val in helplist:
    #           w.writerow([val])


    #     model.save('/home/aleksandar/new_sets/1_5s/results/model_1_5s_64_128.h5')
        #-----------------------

    else:
        print('Using real-time data augmentation.')

        # this will do preprocessing and realtime data augmentation
        datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=False)  # randomly flip images

        # compute quantities required for featurewise normalization
        # (std, mean, and principal components if ZCA whitening is applied)
        datagen.fit(X_train)

        # fit the model on the batches generated by datagen.flow()
        hist = model.fit_generator(datagen.flow(X_train, Y_train,
                            batch_size=batch_size),
                            samples_per_epoch=X_train.shape[0],
                            nb_epoch=nb_epoch,
                            validation_data=(X_test, Y_test))


        #NEW--------------------
        y_pred = model.predict_classes(X_test)
        print(y_pred)
        target_names = ['ArmFasterTowards', 'ArmSlowerTowards', 'CirclingArm', 'Clapping', 'PickingUp', 'Sitting', 'Walking']
    #     sys.stdout = open('/home/aleksandar/sets/cifar_noninitialised_A/results/report_noninitialised_A_2_4.txt', "w")
        print("CLASSIFICATION REPORT:")
        print(classification_report(np.argmax(Y_test,axis=1), y_pred,target_names=target_names))
        print ("\n")
        print("CONFUSION MATRIX:")
        print(confusion_matrix(np.argmax(Y_test,axis=1), y_pred))

    #     with open('/home/aleksandar/sets/cifar_noninitialised_A/results/graphs_noninitialised_A_2_4.csv', "w") as f:
    #        w = csv.writer(f)
    #        train_loss = hist.history['loss']
    #        val_loss = hist.history['val_loss']
    #        train_acc = hist.history['acc']
    #        val_acc = hist.history['val_acc']
    #        helplist = list(zip(train_loss, train_acc, val_loss, val_acc))
    #        w.writerow("train_loss, train_acc, val_loss, val_acc")
    #        for val in helplist:
    #           w.writerow([val])


    #     model.save('/home/aleksandar/sets/cifar_noninitialised_A/results/model_noninitialised_A_2_4.h5')
        #-----------------------

X_train shape: (34720, 1, 75, 75)
34720 train samples
6944 test samples
Not using data augmentation.
Train on 34720 samples, validate on 6944 samples
Epoch 1/2
34720/34720 [==============================] - 77s 2ms/step - loss: 0.7497 - acc: 0.7010 - val_loss: 1.2886 - val_acc: 0.6813
Epoch 2/2
34720/34720 [==============================] - 75s 2ms/step - loss: 0.0484 - acc: 0.9832 - val_loss: 1.5640 - val_acc: 0.6973
[6 0 6 ... 0 6 2]
CLASSIFICATION REPORT:
                  precision    recall  f1-score   support

ArmFasterTowards       0.93      0.83      0.88       992
ArmSlowerTowards       0.57      0.71      0.63       992
     CirclingArm       0.36      0.45      0.40       992
        Clapping       1.00      0.17      0.29       992
       PickingUp       0.82      0.73      0.77       992
         Sitting       0.64      1.00      0.78       992
         Walking       0.99      0.99      0.99       992

     avg / total       0.76      0.70      0.68      6944



CONFUSION 